# Convolutional Neural Network - Cats & Dogs Dataset

## Import & Load Data

In [1]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

--2023-08-07 04:54:24--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.218.128, 142.251.31.128, 142.251.18.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.218.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M  29.0MB/s    in 2.3s    

2023-08-07 04:54:27 (29.0 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import zipfile

local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall()

zip_ref.close()

## Add directory path to accesible variables

In [3]:
import os

base_dir = 'cats_and_dogs_filtered'

print('CONTENTS OF THE BASE DIRECTORY:')
print(os.listdir(base_dir))

print('\nCONTENTS OF TRAINING DIRECTROY:')
print(os.listdir(f'{base_dir}/train'))

print('\nCONTENTS OF VALIDATION DIRECTORY:')
print(os.listdir(f'{base_dir}/validation'))

CONTENTS OF THE BASE DIRECTORY:
['train', 'validation', 'vectorize.py']

CONTENTS OF TRAINING DIRECTROY:
['cats', 'dogs']

CONTENTS OF VALIDATION DIRECTORY:
['cats', 'dogs']


In [4]:
training_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(training_dir, 'cats')
train_dogs_dir = os.path.join(training_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [5]:
print('Training samples - cats', os.listdir(train_cats_dir)[:5])
print('Training samples - dogs', os.listdir(train_dogs_dir)[:5])

Training samples - cats ['cat.560.jpg', 'cat.424.jpg', 'cat.245.jpg', 'cat.711.jpg', 'cat.599.jpg']
Training samples - dogs ['dog.247.jpg', 'dog.521.jpg', 'dog.692.jpg', 'dog.662.jpg', 'dog.802.jpg']


In [7]:
print('Total training cat images:', len(os.listdir(train_cats_dir)))
print('Total training dog images:', len(os.listdir(train_dogs_dir)))

print('Total validation cat images:', len(os.listdir(validation_cats_dir)))
print('Total validation cat images:', len(os.listdir(validation_dogs_dir)))

Total training cat images: 1000
Total training dog images: 1000
Total validation cat images: 500
Total validation cat images: 500


## Define CNN Architecture

In [8]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('cats') and 1 for the other ('dogs')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 64)       0

In [10]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics = ['accuracy'])

## Define traiing and validation generators

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(rescale = 1.0/255.0)
validation_datagen = ImageDataGenerator(rescale = 1.0/255.0)

#define generators
train_generator = train_datagen.flow_from_directory(
    training_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (150,150)
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size = 20,
    class_mode = 'binary',
    target_size = (150,150)
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


## Fit the model

In [15]:
# Fit model over generators
model.fit(train_generator, epochs = 15, validation_data= validation_generator, verbose = 2)

Epoch 1/15
100/100 - 90s - loss: 0.7348 - accuracy: 0.5335 - val_loss: 0.6796 - val_accuracy: 0.5120 - 90s/epoch - 898ms/step
Epoch 2/15
100/100 - 92s - loss: 0.6640 - accuracy: 0.6110 - val_loss: 0.6234 - val_accuracy: 0.6470 - 92s/epoch - 922ms/step
Epoch 3/15
100/100 - 72s - loss: 0.6055 - accuracy: 0.6750 - val_loss: 0.5748 - val_accuracy: 0.6880 - 72s/epoch - 725ms/step
Epoch 4/15
100/100 - 69s - loss: 0.5581 - accuracy: 0.7245 - val_loss: 0.5558 - val_accuracy: 0.6990 - 69s/epoch - 692ms/step
Epoch 5/15
100/100 - 71s - loss: 0.5077 - accuracy: 0.7545 - val_loss: 0.5674 - val_accuracy: 0.7110 - 71s/epoch - 706ms/step
Epoch 6/15
100/100 - 72s - loss: 0.4557 - accuracy: 0.7750 - val_loss: 0.5348 - val_accuracy: 0.7320 - 72s/epoch - 716ms/step
Epoch 7/15
100/100 - 72s - loss: 0.4027 - accuracy: 0.8140 - val_loss: 0.5860 - val_accuracy: 0.7110 - 72s/epoch - 720ms/step
Epoch 8/15
100/100 - 80s - loss: 0.3377 - accuracy: 0.8505 - val_loss: 0.5617 - val_accuracy: 0.7160 - 80s/epoch - 800

## Predictions

In [19]:
import numpy as np

from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded=files.upload()

for fn in uploaded.keys():

  # predicting images
  path='/content/' + fn
  img=load_img(path, target_size=(150, 150))

  x=img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")


Saving vander-films-aPvB8KMIh5w-unsplash.jpg to vander-films-aPvB8KMIh5w-unsplash.jpg
1/1 [==============================] - 0s 159ms/step
[0.6220978]
vander-films-aPvB8KMIh5w-unsplash.jpg is a dog
